In [14]:
import os
import glob
import numpy as np
import pandas as pd
from functions import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
os.chdir("/Users/pestephan/Documents/GitHub/Machine-Learning-using-Sports-Data/Predicting-PremLeague-Outcomes-With-ML/files/Datasets")

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/mycoo/OneDrive/Documents/SportsAnalytics/Machine-Learning-with-the-NFL/Predicting-PremLeague-Outcomes-With-ML/files/Datasets'

Data Taken from http://www.football-data.co.uk/englandm.php

In [2]:
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

In [3]:
#combine all files in the list
master_sheet = pd.concat([pd.read_csv(f) for f in all_filenames ])
#export to csv
master_sheet.to_csv( "master_sheet.csv", index=False, encoding='utf-8-sig')

Reduce table to using columns needed for the model

Columns Required:

Date, Time HomeTeam, AwayTeam, FullTimeHG, FullTimeAG, FullTimeRresults, HalfTimeHG, HalfTimeAG, HomeShots, AwayShots, HomeShotsTarger, AwayShotsTarger, HomeCorners, AwayCorners, HomeFouls, AwayFouls, HomeRed, AwayRed

Columns To Consider:

Attendance, Referee

In [4]:
subset_master = master_sheet[["Date", "Time", "HomeTeam", "AwayTeam", "FTHG", "FTAG", "FTR", "HTHG", "HTAG", "HS", "AS", "HST", "AST", "HC", "AC", "HF", "AF", "HR", "AR"]]
subset_master = subset_master.fillna(0)
# 0 = Home Wins, 1 = Draw, 2 = Away Wins
subset_master.FTR.replace(['H', 'D', 'A'], [0, 1, 2], inplace = True)

In [5]:
subset_master.head()

,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HS,AS,HST,AST,HC,AC,HF,AF,HR,AR
0,14/08/10,0,Aston Villa,West Ham,3.0,0.0,0,2.0,0.0,23.0,12.0,11.0,2.0,16.0,7.0,15.0,15.0,0.0,0.0
1,14/08/10,0,Blackburn,Everton,1.0,0.0,0,1.0,0.0,7.0,17.0,2.0,12.0,1.0,3.0,19.0,14.0,0.0,0.0
2,14/08/10,0,Bolton,Fulham,0.0,0.0,1,0.0,0.0,13.0,12.0,9.0,7.0,4.0,8.0,12.0,13.0,0.0,0.0
3,14/08/10,0,Chelsea,West Brom,6.0,0.0,0,2.0,0.0,18.0,10.0,13.0,4.0,3.0,1.0,10.0,10.0,0.0,0.0
4,14/08/10,0,Sunderland,Birmingham,2.0,2.0,1,1.0,0.0,6.0,13.0,2.0,7.0,3.0,6.0,13.0,10.0,1.0,0.0


In [6]:
msk = np.random.rand(len(subset_master)) < 0.70

train_df = subset_master[msk]
test_df = subset_master[~msk]

X_train = train_df.drop(columns = ['Date', 'Time', 'HomeTeam', 'AwayTeam', 'FTR'])
y_train = train_df[['FTR']] 
X_test = test_df.drop(columns = ['Date', 'Time', 'HomeTeam', 'AwayTeam', 'FTR'])
y_test = test_df[['FTR']]

In [11]:
clf = LogisticRegression(penalty='l1', dual=False, tol=0.001, C=1.0, fit_intercept=True, 
                   intercept_scaling=1, class_weight='balanced', random_state=None, 
                   solver='liblinear', max_iter=10000, multi_class='ovr', verbose=0)

clf.fit(X_train, np.ravel(y_train.values))

y_pred = clf.predict_proba(X_test)
y_pred = pd.DataFrame(y_pred)
subset_master = subset_master.dropna()

,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HS,...,AC,HF,AF,HR,AR,HomeWinPerc,DrawPerc,AwayPerc,PredResult,Comparison
0,14/08/10,0,Aston Villa,West Ham,3.0,0.0,0,2.0,0.0,23.0,...,7.0,15.0,15.0,0.0,0.0,7.696774e-01,0.230323,1.376141e-17,H,False
1,14/08/10,0,Blackburn,Everton,1.0,0.0,0,1.0,0.0,7.0,...,3.0,19.0,14.0,0.0,0.0,5.291631e-03,0.989439,5.269168e-03,D,False
2,14/08/10,0,Bolton,Fulham,0.0,0.0,1,0.0,0.0,13.0,...,8.0,12.0,13.0,0.0,0.0,3.979148e-08,0.216988,7.830124e-01,A,False
3,14/08/10,0,Chelsea,West Brom,6.0,0.0,0,2.0,0.0,18.0,...,1.0,10.0,10.0,0.0,0.0,7.065285e-01,0.293471,1.047134e-07,H,False
4,14/08/10,0,Sunderland,Birmingham,2.0,2.0,1,1.0,0.0,6.0,...,6.0,13.0,10.0,1.0,0.0,1.302891e-32,0.149665,8.503349e-01,A,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20614,05/11/11,0,Newcastle,Everton,2.0,1.0,0,2.0,1.0,10.0,...,3.0,12.0,15.0,0.0,0.0,3.129357e-08,0.309748,6.902521e-01,A,False
20615,05/11/11,0,QPR,Man City,2.0,3.0,2,1.0,1.0,18.0,...,1.0,5.0,14.0,0.0,0.0,6.301988e-09,0.346677,6.533228e-01,A,False
20616,06/11/11,0,Bolton,Stoke,5.0,0.0,0,2.0,0.0,20.0,...,4.0,10.0,9.0,0.0,0.0,5.435298e-02,0.926161,1.948588e-02,D,False
20617,06/11/11,0,Fulham,Tottenham,1.0,3.0,2,0.0,2.0,26.0,...,1.0,5.0,9.0,0.0,0.0,5.632468e-18,0.155745,8.442551e-01,A,False


In [13]:
metrics(subset_master, y_pred)

False    100.0
Name: Comparison, dtype: float64
0    45.230999
2    30.228515
1    24.540487
Name: FTR, dtype: float64
0    63.991409
H    16.541013
A    10.703954
D     8.763625
Name: PredResult, dtype: float64


In [14]:
subset_master

,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HS,...,AC,HF,AF,HR,AR,HomeWinPerc,DrawPerc,AwayPerc,PredResult,Comparison
0,14/08/10,0,Aston Villa,West Ham,3.0,0.0,0,2.0,0.0,23.0,...,7.0,15.0,15.0,0.0,0.0,7.689865e-01,0.231014,5.084868e-17,H,False
1,14/08/10,0,Blackburn,Everton,1.0,0.0,0,1.0,0.0,7.0,...,3.0,19.0,14.0,0.0,0.0,6.921051e-03,0.987593,5.485749e-03,D,False
2,14/08/10,0,Bolton,Fulham,0.0,0.0,1,0.0,0.0,13.0,...,8.0,12.0,13.0,0.0,0.0,5.042411e-08,0.216830,7.831699e-01,A,False
3,14/08/10,0,Chelsea,West Brom,6.0,0.0,0,2.0,0.0,18.0,...,1.0,10.0,10.0,0.0,0.0,7.063584e-01,0.293642,1.242519e-07,H,False
4,14/08/10,0,Sunderland,Birmingham,2.0,2.0,1,1.0,0.0,6.0,...,6.0,13.0,10.0,1.0,0.0,7.418114e-32,0.149312,8.506880e-01,A,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64411,07/02/2021,12:00,Tottenham,West Brom,2.0,0.0,0,0.0,0.0,13.0,...,0.0,16.0,12.0,0.0,0.0,NaN,NaN,NaN,0,False
64412,07/02/2021,14:00,Wolves,Leicester,0.0,0.0,1,0.0,0.0,13.0,...,2.0,11.0,15.0,0.0,0.0,NaN,NaN,NaN,0,False
64413,07/02/2021,16:30,Liverpool,Man City,1.0,4.0,2,0.0,0.0,8.0,...,1.0,13.0,8.0,0.0,0.0,NaN,NaN,NaN,0,False
64414,07/02/2021,19:15,Sheffield United,Chelsea,1.0,2.0,2,0.0,1.0,8.0,...,8.0,8.0,8.0,0.0,0.0,NaN,NaN,NaN,0,False
